In [1]:
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
from tf_pose import common
import subprocess
import gc
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from analyze.analyze_human import AnalyzeHuman
%matplotlib inline

w, h = model_wh('432x368')
estimator = TfPoseEstimator(get_graph_path('cmu'), target_size=(w, h))
analyzeHuman = AnalyzeHuman()

/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

[2020-12-19 05:18:53,530] [TfPoseEstimator] [INFO] loading graph from /usr/src/app/tf_pose_estimation/models/graph/cmu/graph_opt.pb(default size=432x368)
2020-12-19 05:18:53,530 INFO loading graph from /usr/src/app/tf_pose_estimation/models/graph/cmu/graph_opt.pb(default size=432x368)


2020-12-19 05:18:53,534 WARNING From /usr/src/app/tf_pose_estimation/tf_pose/estimator.py:311: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



2020-12-19 05:18:53,540 WARNING From /usr/src/app/tf_pose_estimation/tf_pose/estimator.py:312: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef instead.



2020-12-19 05:18:54,012 WARNING From /usr/src/app/tf_pose_estimation/tf_pose/estimator.py:330: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



...

2020-12-19 05:18:55,166 WARNING From /usr/src/app/tf_pose_estimation/tf_pose/estimator.py:341: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



2020-12-19 05:18:55,182 WARNING From /usr/src/app/tf_pose_estimation/tf_pose/estimator.py:342: The name tf.image.resize_area is deprecated. Please use tf.compat.v1.image.resize_area instead.



2020-12-19 05:18:55,242 WARNING From /usr/src/app/tf_pose_estimation/tf_pose/tensblur/smoother.py:96: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2020-12-19 05:18:55,281 WARNING From /usr/src/app/tf_pose_estimation/tf_pose/estimator.py:354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [2]:
video_name = 'dance'
extension = '.mp4'
path_video = f'./etcs/{video_name}{extension}'

path_image_out_dir = os.path.join(f'./outputs/estimated_images/{video_name}/')
path_video_out_dir = os.path.join('./outputs/estimated_videos/')
path_data_out_dir = './outputs/data/'
path_data_out = f'./outputs/data/{video_name}.xlsx'

os.makedirs(path_image_out_dir, exist_ok=True)
os.makedirs(path_video_out_dir, exist_ok=True)
os.makedirs(path_data_out_dir, exist_ok=True)

cap = cv2.VideoCapture(path_video)

if not cap.isOpened():
    print('not found')

In [3]:
start_frame = 0
# end_frame = 305

frame_no = 0
while cap.isOpened():
    ret_val, image = cap.read()
    if not ret_val:
        break
    if frame_no < start_frame:
        frame_no += 1
        continue
#     if frame_no > end_frame:
#         break
    print(f'processing... frame no={frame_no}')
    humans = estimator.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=4.0)
    estimated_image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
    fig = plt.figure()
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.imsave(path_image_out_dir+f'{frame_no}.png', estimated_image)
    plt.clf()
    plt.close()
    
    if len(humans) >= 1:
        human_points_df = analyzeHuman.get_and_append_human_points(humans[0], 
                                                                   frame_no, 
                                                                   estimated_image.shape[0], 
                                                                   estimated_image.shape[1])
    else:
        human_points_df = analyzeHuman.append_none_human_points(frame_no)
    
    frame_no += 1
    gc.collect()

processing... frame no=300
processing... frame no=301
processing... frame no=302
processing... frame no=303
processing... frame no=304
processing... frame no=305


In [5]:
# !rm {path_video_out_dir + video_name + extension}
cmd = ['ffmpeg', '-framerate', str(cap.get(cv2.CAP_PROP_FPS)), 
       '-start_number', str(start_frame),
       '-i', os.path.join(path_image_out_dir, '%d.png'),
       '-vcodec', 'libx264', '-pix_fmt', 'yuv420p',
        os.path.join(path_video_out_dir, video_name + extension)]
subprocess.call(cmd)

0

In [6]:
analyzeHuman.save_points(path_data_out)

In [8]:
analyzeHuman._df.head()

,frame_name,Nose_x,Nose_y,Nose_score,Neck_x,Neck_y,Neck_score,RShoulder_x,RShoulder_y,RShoulder_score,...,REye_score,LEye_x,LEye_y,LEye_score,REar_x,REar_y,REar_score,LEar_x,LEar_y,LEar_score
0,300,220.000000,194.782609,0.758662,238.333333,285.217391,0.817789,211.666667,281.739130,0.787456,...,0.778486,226.666667,177.391304,0.855072,,,,245.000000,177.391304,0.784310
1,301,215.000000,194.782609,0.890981,235.000000,285.217391,0.847748,208.333333,285.217391,0.828976,...,0.714547,221.666667,177.391304,0.868228,,,,240.000000,177.391304,0.809105
2,302,116.666667,177.391304,0.790121,123.333333,306.086957,0.720084,86.666667,306.086957,0.609903,...,0.910366,126.666667,156.521739,0.827669,,,,143.333333,163.478261,0.838757
3,303,203.333333,205.217391,0.906976,225.000000,292.173913,0.750389,198.333333,295.652174,0.790954,...,0.695644,208.333333,184.347826,0.960838,,,,226.666667,180.869565,0.776708
4,304,195.000000,208.695652,0.831180,218.333333,292.173913,0.742329,193.333333,299.130435,0.735875,...,0.803850,201.666667,187.826087,0.918347,,,,218.333333,184.347826,0.798382
